# Twitter y MongoDB

De las múltiples librerías que nos permiten usar la API de Twitter, usaremos [Python Twitter Tools](https://github.com/sixohsix/twitter). Con esta librería podremos descargar tweets e información de sus usuarios, así que el **ejercicio** será modelar estas dos entidades y almacenar instancias de ellas.

La idea de este ejercicio está basada en uno anterior que realizó [Gabriel Muñoz](https://twitter.com/Gabi_mu_ri).

## Requisitos

* Python:
  * `jupyter`
  * [`twitter`](https://github.com/sixohsix/twitter)
  * `pymongo`
  * `mongoengine`
* MongoDB

## El modelo

Más o menos todos tenemos en la cabeza como funciona Twitter:
* Un usuario puede publicar cero o muchos tweets.
* Un tweet tienes varias propiedades, siendo una de ellas el tweet en si, donde podemos encontrar:
  * Texto.
  * Menciones a otros usuarios: [`@josemazo`](https://twitter.com/josemazo).
  * Enlaces: [`https://www.mongodb.org/`](https://www.mongodb.org/).
  * Hashtags: `#MongoDB`.

## Autorización

Para usar la API pública de Twitter necesitamos ciertos parámetros, así que vamos a ver como obtenerlos.
1. Debemos tener una cuenta en Twitter y estar logueados.
2. Visitamos [https://apps.twitter.com/](https://apps.twitter.com/) y pulsamos sobre **`Create New App`**.
![Pasos 1 y 2](https://i.imgur.com/85p8ROC.png)
3. Rellenamos los campos obligatorios:
  * **`Name`**: debe ser único.
  * **`Description`**: debe tener más de 10 carácteres.
  * **`Website`**: debe ser una URL válida.
  * **`Yes, I agree`**: debemos marcar ese checkbox.
  * Finalmente puslamos sobre **`Create your Twitter application`**.
![Paso 3](https://i.imgur.com/Yi3vATT.png)
4. Pulsamos en el enlace **`Keys and Access Tokens`**.
![Paso 4](https://i.imgur.com/BVbSK4M.png)
5. En `Application Settings` tenemos dos de los parámetros que necesitamos:
  * **`Consumer Key`**
  * **`Consumer Secret`**
![Paso 5](https://i.imgur.com/R0UQFTq.png)
6. Al final de la misma página hay un botón que dice **`Create my access token`**, pulsamos sobre él.
![Paso 6](https://i.imgur.com/x6HO5Wy.png)
7. De nuevo al final de esa misma página, bajo `Your Access Token` tenemos los otros dos parámetros restantes:
  * **`Access Token`**
  * **`Access Token Secret`**
![Paso 7](https://i.imgur.com/oNvj1z5.png)

## ¡A programar!

In [42]:
# Importing packages
from bson.objectid import ObjectId
import datetime
from mongoengine import *
import twitter

In [43]:
# Twitter configuration, we need to use the parameters that we got before
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [44]:
# MongoDB configuration
connect('twitter', host='localhost', port=27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

Vamos a obtener tweets, para ellos usaremos la API y veremos que nos devuelve.

In [45]:
# Obtaining tweets
search_word = '#goya2017'  # We can search by a word, a text or a hashtag
count = 200

search_results = twitter_api.search.tweets(q=search_word, count=count, lang='es')
search_results

{'search_metadata': {'completed_in': 0.118,
  'count': 100,
  'max_id': 828304366885076997,
  'max_id_str': '828304366885076997',
  'next_results': '?max_id=828303440543612927&q=%23goya2017&lang=es&count=100&include_entities=1',
  'query': '%23goya2017',
  'refresh_url': '?since_id=828304366885076997&q=%23goya2017&lang=es&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'},
 'statuses': [{'contributors': None,
   'coordinates': None,
   'created_at': 'Sun Feb 05 18:08:44 +0000 2017',
   'entities': {'hashtags': [{'indices': [30, 39], 'text': 'Goya2017'}],
    'media': [{'display_url': 'pic.twitter.com/Zxcl9EAMvJ',
      'expanded_url': 'https://twitter.com/Aquel_Coche/status/828000811263393793/photo/1',
      'id': 828000805970182145,
      'id_str': '828000805970182145',
      'indices': [40, 63],
      'media_url': 'http://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
      'media_url_https': 'https://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
      'sizes': {'large': {'h': 399, 

In [46]:
search_results['statuses']

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Sun Feb 05 18:08:44 +0000 2017',
  'entities': {'hashtags': [{'indices': [30, 39], 'text': 'Goya2017'}],
   'media': [{'display_url': 'pic.twitter.com/Zxcl9EAMvJ',
     'expanded_url': 'https://twitter.com/Aquel_Coche/status/828000811263393793/photo/1',
     'id': 828000805970182145,
     'id_str': '828000805970182145',
     'indices': [40, 63],
     'media_url': 'http://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
     'sizes': {'large': {'h': 399, 'resize': 'fit', 'w': 599},
      'medium': {'h': 399, 'resize': 'fit', 'w': 599},
      'small': {'h': 399, 'resize': 'fit', 'w': 599},
      'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
     'source_status_id': 828000811263393793,
     'source_status_id_str': '828000811263393793',
     'source_user_id': 3380960747,
     'source_user_id_str': '3380960747',
     'type': 'photo',
     'url': 'https://t.

In [47]:
search_results['statuses'][0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Feb 05 18:08:44 +0000 2017',
 'entities': {'hashtags': [{'indices': [30, 39], 'text': 'Goya2017'}],
  'media': [{'display_url': 'pic.twitter.com/Zxcl9EAMvJ',
    'expanded_url': 'https://twitter.com/Aquel_Coche/status/828000811263393793/photo/1',
    'id': 828000805970182145,
    'id_str': '828000805970182145',
    'indices': [40, 63],
    'media_url': 'http://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/C32mWJmXAAEZy0f.jpg',
    'sizes': {'large': {'h': 399, 'resize': 'fit', 'w': 599},
     'medium': {'h': 399, 'resize': 'fit', 'w': 599},
     'small': {'h': 399, 'resize': 'fit', 'w': 599},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 828000811263393793,
    'source_status_id_str': '828000811263393793',
    'source_user_id': 3380960747,
    'source_user_id_str': '3380960747',
    'type': 'photo',
    'url': 'https://t.co/Zxcl9EAMvJ'}],
  '

In [48]:
search_results['statuses'][0]['user']

{'contributors_enabled': False,
 'created_at': 'Mon Feb 08 17:33:26 +0000 2016',
 'default_profile': True,
 'default_profile_image': False,
 'description': '',
 'entities': {'description': {'urls': []}},
 'favourites_count': 1660,
 'follow_request_sent': False,
 'followers_count': 208,
 'following': False,
 'friends_count': 915,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 4889145250,
 'id_str': '4889145250',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'es',
 'listed_count': 11,
 'location': '',
 'name': 'buenprovecho',
 'notifications': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': None,
 'profile_background_image_url_https': None,
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4889145250/1454953180',
 'profile_image_url': 'http://pbs.twimg.com/profile_images/696749674661867520/qu2HwUKz_normal.jpg',
 'profile_image_url_https': 'https://pbs.twimg.com/profile_image

Vamos a crear nuestros modelos. Primero haremos uno para los tweets, donde podremos utilizar las propiedades que querramos, pero por ejemplo, podriamos usar los siguientes:
* `created_at`
* `text`
* `retweet_count`
* `favorite_count`
* `hashtags`
* `urls`
* `mentions`
* `user_id`

Para los usuarios podríamos usar algo así:
* `created_at`
* `screen_name`
* `name`
* `description`
* `favourites_count`
* `followers_count`
* `friends_count`
* `profile_image_url`

Algunos de ellos no existen como tal en los resultados de la API, pero para ello podremos programar funciones que nos transformen los resultados en datos válidos para el modelo.

In [49]:
class Tweet(Document):
    ######################################################
    # Here we need a block for the properties
    ######################################################
    # Set some fields with default and required parameters
    ######################################################
    user_id = ObjectIdField(required=True)
    created_at = DateTimeField(required=True)
    text = StringField()
    retweet_count = IntField(required=True)
    favorite_count = IntField(required=True)
    hashtags = ListField(StringField())
    urls = ListField(URLField())
    mentions = ListField()
    

class User(Document):
    ######################################################
    # Here we need a block for the properties
    ######################################################
    # Set some fields with default and required parameters
    ######################################################
    created_at = DateTimeField(required=True)
    screen_name = StringField(max_length=50, required=True)
    name = StringField()
    description = StringField()
    favourites_count = IntField(required=True)
    followers_count = IntField(required=True)
    friends_count = IntField(required=True)
    profile_image_url = URLField()
    location = StringField()

In [50]:
# Function for transform the datetime from Twitter to Python's format
def twitter_date_to_datetime(twitter_date):
    return datetime.datetime.strptime(twitter_date, '%a %b %d %H:%M:%S +0000 %Y')

## Constructors for the documents
# User getter and constructor
def get_or_create_user(api_user):
    user = User.objects(screen_name=api_user['screen_name']).first()
    if not user:
        user = User()
        
        user['created_at'] = twitter_date_to_datetime(api_user['created_at'])
        user['screen_name'] = api_user['screen_name']
        ###############################################
        # Here we need a block for link more properties
        ###############################################
        user['name'] = api_user['name']  
        user['description'] = api_user['description']
        user['favourites_count'] = api_user['favourites_count']
        user['followers_count'] = api_user['followers_count']
        user['friends_count'] = api_user['friends_count']
        user['profile_image_url'] = api_user['profile_image_url']
        user['location'] = api_user['location']
        ##################################
        # And now we must persist the user
        ##################################
        user.save()
    
    return user

# Tweet constructor
def create_tweet(api_tweet, user):
    tweet = Tweet()
    
    tweet['created_at'] = twitter_date_to_datetime(api_tweet['created_at'])
    tweet['text'] = api_tweet['text']
    ###############################################
    # Here we need a block for link more properties
    ###############################################

    hashtags = [hashtag['text'] for hashtag in api_tweet['entities']['hashtags']]
    urls = [url['expanded_url'] for url in api_tweet['entities']['urls']]
    mentions = [mention['screen_name'] for mention in api_tweet['entities']['user_mentions']]
    
    tweet['hashtags'] = hashtags
    tweet['urls'] = urls
    tweet['mentions'] = mentions
    ###############################################
    # Here we need a block for link more properties
    ###############################################
    tweet['retweet_count'] = api_tweet['retweet_count']
    tweet['favorite_count'] = api_tweet['favorite_count']
    
    tweet['user_id'] = user.id
    
    ###################################
    # And now we must persist the tweet
    ###################################
    tweet.save()
    
    
    return tweet

Por último, vamos a rellenar nuestra base de datos con los resultados obtenidos y comprobar en RoboMongo los resultados.

In [51]:
for result in search_results['statuses']:
    api_user = result['user']
    user = get_or_create_user(api_user)
    tweet = create_tweet(result, user)